# Introspecting objects

Remember `dir()` is another important introspection function, which returns a list of attribute names for an instance:

In [ ]:
a = 42

In [ ]:
dir(a)

In [ ]:
b = "Donovan"

In [ ]:
dir(b)

In [ ]:
b.upper()

Both attributes and method names are returned in the above lists, because methods _are_ attributes - they are just callable attributes.

## `getattr()`

Given an object and an attribute name, a corresponding attribute object using the built-in `getattr()` function.  See that the `int` object has attributes called `numerator` and `denominator` allowing it to be used like the rational number object modelled by the `Fraction` type, and also `imag`, `real`, and `conjugate` attributes allowing it to be used like a complex number.  Retrieve one of the attributes using `getattr()`:

In [ ]:
getattr(a, 'denominator')

The above returns the same value as accessing the denominator directly:

In [ ]:
a.denominator

Other attributes return more interesting objects:

In [ ]:
getattr(a, 'conjugate')

The `conjugate` attribute is revealed to be a method.  Check that it is a callable object using another introspection tool, the built-in `callable()` function:

In [ ]:
callable(getattr(a, 'conjugate'))

In [ ]:
callable(getattr(b, 'isdecimal'))

By navigating around the attributes of an object, and the attributes of those attributes, it is possible to discover almost anything about an object in Python.  Retrieve the type of the `int.conjugate` method through its `__class__` attribute, and then the `__name__` of that class as a string through the `__name__` attribute.

## `hasattr()`

It can be determined whether a particular object has an attribute of a given name using the built-in `hasattr()` function, which returns True if a particular attribute exists:

In [ ]:
hasattr(a, 'bit_length')

In [ ]:
hasattr(a, 'index')

In [ ]:
hasattr(b, 'index')

In general, _Easier to Ask for Forgiveness than Permission_ (EAFP) style programming using exception handlers is considered more Pythonic than _Look Before You Leap_ (LBLY) style programs using type test and attribute existence tests.  Optimistic code using `try..except` is faster that LBLY code using `hasattr()`, becuase internally `hasattr()` uses an exception handler anyway.

In [ ]:
# The Zen of Python
import this

Here is a function in a module `numerals.py` which, given an object supporting the `numerator` and `denominator` attributes for rational numbers, returns a so-called mixed-numeral containing the separate whole number and fractional parts:

In [ ]:
# numerals.py

from fractions import Fraction

def mixed_numeral(vulgar):
    if not (hasattr(vulgar, 'numerator') and hasattr(vulgar, 'denominator')):
        raise TypeError("{} is not a rational number".format(vulgar))

    integer = vulgar.numerator // vulgar.denominator
    fraction = Fraction(vulgar.numerator - integer * vulgar.denominator, vulgar.denominator)
    return integer, fraction

The above version uses `hasattr()` to check whether the supplied object supports the rational number interface.  Call it with a `Fraction`:

In [ ]:
mixed_numeral(Fraction('11/10'))

Call it with a float, and see that the type check fails:

In [ ]:
mixed_numeral(1.7)

In contrast, here is an alternative version which does not perform the `hasattr()` check:

In [ ]:
def mixed_numeral(vulgar):
    integer = vulgar.numerator // vulgar.denominator
    fraction = Fraction(vulgar.numerator - integer * vulgar.denominator, vulgar.denominator)
    return integer, fraction

The only difference in its runtime behavior is that a different exception is raised:

In [ ]:
mixed_numeral(Fraction('11/10'))

In [ ]:
mixed_numeral(1.7)

The above exception is more detailed, but certainly less informative.  Of course an exception handler can be used to raise the more appropriate exception type of `TypeError` chaining it to the original `AttributeError` to provide the details:

In [ ]:
def mixed_numeral(vulgar):
    try:
        integer = vulgar.numerator // vulgar.denominator
        fraction = Fraction(vulgar.numerator - integer * vulgar.denominator, vulgar.denominator)
        return integer, fraction
    except AttributeError as e:
        raise TypeError("{} is not a rational number".format(vulgar)) from e

The above approach yields the maximum amount of information about what went wrong and why:

In [ ]:
mixed_numeral(Fraction('11/10'))

In [ ]:
mixed_numeral(1.7)

The lesson is that if using type introspection in Python it may be better to avoid the checks and just "let it fail".